1. Compare detection with different languages, configs, resolution
Get config impact with varying dpi
Get language impact with varying dpi
Get image enhancement impact with varying dpi, for default and pol language 

In [1]:
import os

In [2]:
os.chdir('./..')
os.getcwd()

'C:\\Users\\Michal\\Documents\\GitHub\\OCR-Receipts'

In [3]:
import cv2
import pytesseract

In [11]:
from scripts.content_detector import process_image
process_image.LOG = False

## Read raw image

In [5]:
os.listdir('images/receipts')

['Paragon_2022-08-11_081037_150dpi.jpg',
 'Paragon_2022-08-11_081131_300dpi.jpg',
 'Paragon_2022-08-11_081332_600dpi.jpg']

In [107]:
raw_img_folderpath = 'images/receipts'

In [108]:
# Get raw images
raw_imgs = dict.fromkeys([150, 300, 600])
for dpi, fname in zip(raw_imgs, os.listdir(raw_img_folderpath)):
    raw_img_filepath = os.path.join(raw_img_folderpath, fname)
    raw_imgs[dpi] = process_image.read_image(raw_img_filepath)

In [109]:
# Show raw images
for dpi, img in raw_imgs.items():
    cv2.imshow(f'{dpi} dpi', img)

cv2.waitKey()

-1

In [111]:
# Get content from raw images
raw_contents = {}
for dpi, img in raw_imgs.items():
    raw_contents[dpi] = pytesseract.image_to_string(cv2.cvtColor(img, cv2.COLOR_BGR2RGB),
                                                   config='--psm 4')

In [112]:
# Print the raw content
raw_contents_split = {dpi: content.split('\n') for dpi, content in raw_contents.items()}

for line_150, line_300, line_600 in zip(*raw_contents_split.values()):
    print(f'{line_150:<40}|{line_300:<40}|{line_600:<40}')

a “"CODZIENNIE NISKIE CENY” 4884        |BIEDRONKA “CODZIENNIE NISKIE CENY” 4884 |BIEDRONKA “CODZTENNTE NISKIE CENY” dead 
0 WARSZAWA UL. BOKSERSKA 64             |02-690 WARSZAWA UL, BOKSERSKA 64        |02-690 WARSZAWA UL, BORSERSKA 64        
"FROM MARTINS POLSKA S.A.               |JERONIMO MARTINS POLSKA S.A.            |JERONIMO MARTINS POLSKA 5.A.            
62-025 KOSTRZYN UL.2NINNA 5             |62-025 KOSTRZYN UL. ZNIHNA 5            |62-025 KOSTRZYN UL.ZNIMNA 5             
                                        |                                        |                                        
NIP 7791044327 ar 185679                |NIP 7794011327 wr 485679                |NIP 7794041327 nr 185679                
pipecor FISKALNY                        |PARAGON FISKALNY                        |PARAGON FISKALNY                        
PassataSot tGus700g i x3,99 3,990       |PassataSot thus 708g b { x3,99 3,990    |PassataSot thus /00g [ 1 x3,99 3,990    
Sos Hadero 250al

300 dpi resolution seems to provide the best results.

## Prepare image
Adjust the image to enhance content recognition.

In [113]:
raw_img = raw_imgs[300]

## Get content

Check the best image resolution for content extraction.